In [4]:
import re

In [24]:
m = re.search(r'(?<=-)\w+', 'spam-egg')
m.group(0)

'egg'

In [34]:
re.sub(r'def\s+([a-zA-Z_][a-zA-Z_0-9]*)\s*\(\s*\):',
       r'static PyObject*\npy_\1(void)\n{',
       'def myfunc():')

'static PyObject*\npy_myfunc(void)\n{'

In [54]:
class GenProp(object):
    def __init__(self, prop_type, prop_name):
        self.prop_type=prop_type
        self.prop_name=prop_name
class GenCollection(object):
    def __init__(self, collection_type, element_type, prop_name):
        self.prop_name=prop_name
        self.collection_type=collection_type
        self.element_type=element_type
class GenMeta(object):
    def __init__(self):
        self.class_name=''
        self.base_name=''
        self.attributes=[]

In [55]:
code="""class Schedule extends Equatable {
  DateTime fromDate= DateTime.now();
  TimeOfDay fromTime = const TimeOfDay(hour: 7, minute: 28);
  DateTime toDate = DateTime.now();
  TimeOfDay toTime = const TimeOfDay(hour: 7, minute: 28);
  // final List<String> allActivities = <String>['hiking', 'swimming', 'boating', 'fishing'];
  List<String> allActivities= <String>['hiking', 'swimming'];
  String activity = 'fishing';
"""
meta=GenMeta()
for line in code.splitlines():    
    # print(line)
    match_obj=re.match(r'class (.*) extends (.*) {', line, re.M|re.I)
    if match_obj:
        print("@class", match_obj.group(1),"*", match_obj.group(2))
        meta.class_name=match_obj.group(1)
        meta.base_name=match_obj.group(2)
    else:
        pass
    
    match_obj=re.match(r'\s*([a-zA-Z_][a-zA-Z_0-9]*)\s*([a-zA-Z_][a-zA-Z_0-9]*)\s*=(.*);', line, re.M|re.I)
    # match_obj=re.search(r'([a-zA-Z_]+) ([a-zA-Z_]+) = ([a-zA-Z_]+);', line)
    if match_obj:
        print("@prop", match_obj.group(1),'*', match_obj.group(2))
        meta.attributes.append(GenProp(match_obj.group(1), match_obj.group(2)))
    else:
        pass
    
    # process collections
    match_obj=re.match(r'\s*([a-zA-Z_][a-zA-Z_0-9]*)<(.*)>\s*([a-zA-Z_][a-zA-Z_0-9]*)\s*=(.*);', line, re.M|re.I)
    if match_obj:
        print("@collection", match_obj.group(1),"-", match_obj.group(2), match_obj.group(3))
        meta.attributes.append(GenCollection(match_obj.group(1),match_obj.group(2), match_obj.group(3)))
    else:
        pass

print(meta)

@class Schedule * Equatable
@prop DateTime * fromDate
@prop TimeOfDay * fromTime
@prop DateTime * toDate
@prop TimeOfDay * toTime
@collection List - String allActivities
@prop String * activity


In [67]:
import json
# json.dumps(meta.__dict__)
from json import JSONEncoder
class MyEncoder(JSONEncoder):
    def default(self, o):
        return o.__dict__    

# data=(MyEncoder().encode(meta))
# print(data)
jd=json.dumps(meta, cls=MyEncoder, indent=2)
print(jd)

{
  "class_name": "Schedule",
  "base_name": "Equatable",
  "attributes": [
    {
      "prop_type": "DateTime",
      "prop_name": "fromDate"
    },
    {
      "prop_type": "TimeOfDay",
      "prop_name": "fromTime"
    },
    {
      "prop_type": "DateTime",
      "prop_name": "toDate"
    },
    {
      "prop_type": "TimeOfDay",
      "prop_name": "toTime"
    },
    {
      "prop_name": "allActivities",
      "collection_type": "List",
      "element_type": "String"
    },
    {
      "prop_type": "String",
      "prop_name": "activity"
    }
  ]
}


In [71]:
def gen_constructor(meta):
    lines=[]
    lines.append('  %s({'%meta.class_name)
    attr_defs=[]
    attr_equals=[]
    for attr in meta.attributes:
        attr_defs.append('this.%s'%attr.prop_name)
        attr_equals.append(attr.prop_name)
    lines.append('    '+', '.join(attr_defs))
    lines.append('  }) : super([')
    lines.append('    '+', '.join(attr_equals))
    lines.append('  ]);')
    print('\n'.join(lines))

gen_constructor(meta)

  Schedule({
    this.fromDate, this.fromTime, this.toDate, this.toTime, this.allActivities, this.activity
  }) : super([
    fromDate, fromTime, toDate, toTime, allActivities, activity
  ]);


In [73]:
lines=[]
lines.append('  static %s overrides%s(Map<String, dynamic> map) {'%(meta.class_name, meta.class_name))
lines.append('    return %s('%meta.class_name)
setters=[]
for attr in meta.attributes:
    setters.append("        %s: map['%s']"%(attr.prop_name, attr.prop_name))
lines.append(",\n".join(setters))
lines.append('  }')
print('\n'.join(lines))

  static Schedule overridesSchedule(Map<String, dynamic> map) {
    return Schedule(
        fromDate: map['fromDate'],
        fromTime: map['fromTime'],
        toDate: map['toDate'],
        toTime: map['toTime'],
        allActivities: map['allActivities'],
        activity: map['activity']
  }


In [77]:
lines=[]
lines.append('  Map<String, dynamic> asMap() {')
setters=[]
for attr in meta.attributes:
    setters.append("'%s':%s"%(attr.prop_name, attr.prop_name))
lines.append("    return {%s};"%(', '.join(setters)))
lines.append('  }')
print('\n'.join(lines))

  Map<String, dynamic> asMap() {
    return {'fromDate':fromDate, 'fromTime':fromTime, 'toDate':toDate, 'toTime':toTime, 'allActivities':allActivities, 'activity':activity};
  }
